# CR6 Results Analysis

## Python Imports

In [9]:
import numpy as np
import pandas as pd
from prettypandas import PrettyPandas
import patsy
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api

from IPython.display import display, Markdown, HTML

%matplotlib inline
PlotWidth = 6

import warnings
#warnings.filterwarnings('ignore')

### Install R Packages

In [2]:
# Using R inside python
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages

# Allow conversion
import rpy2.robjects as ro

from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

# Install packages
packnames = ('rsm')
#utils.install_packages(StrVector(packnames))

# Load packages
rsm = importr('rsm')
base = importr('base')

In [ ]:
# helper functions for this notebook

# use SVG for matplotlib-based figures
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def coded_to_actual(coded_data, actual_lows, actual_highs):
    """Converts a pandas DataFrame from coded units to actuals."""
    actual_data = coded_data.copy()
    for col in actual_data.columns:
        if not (col in actual_highs and col in actual_lows):
            continue
        try:
            # convert continuous variables to their actual value
            actual_data[col] *= 0.5 * (float(actual_highs[col]) - float(actual_lows[col]))
            # don't need to cast to float here, if either are not a float exception will have been thrown
            actual_data[col] += 0.5 * (actual_highs[col] + actual_lows[col])
        except ValueError:
            # assume 2 level categorical
            actual_data[col] = actual_data[col].map({-1: actual_lows[col], 1: actual_highs[col]})
    return actual_data
        
def get_tick_labels(key, lows, highs, units):
    """Returns a list of low/high labels with units (e.g. [8mm, 10mm])"""
    return [str(lows[key]) + units[key], str(highs[key]) + units[key]]

## Process CSV Files

In [3]:

# importing the pandas library
import pandas as pd
  
# reading the csv file using read_csv
# storing the data frame in variable called df
dfcost = pd.read_csv('https://raw.githubusercontent.com/wilsongis/3DP_Experiments/main/Data/cr6_cost.txt', sep='\t')
dftime = pd.read_csv('https://raw.githubusercontent.com/wilsongis/3DP_Experiments/main/Data/cr6_time.txt', sep='\t')
  
# creating a list of column names by
# calling the .columns
list_of_columns_cost = list(dfcost.columns)
list_of_columns_time = list(dftime.columns)
  
# displaying the list of column names
print('List of Cost column names : ', 
      list_of_columns_cost)
print('List of Time column names : ', 
      list_of_columns_time)

List of Cost column names :  ['lh', 'ps', 'id', 'rw', 'wt', 'alh', 'aps', 'aid', 'arw', 'awt', 'cost1', 'cost2', 'cost3', 'cost4', 'cost5']
List of Time column names :  ['lh', 'ps', 'id', 'rw', 'wt', 'alh', 'aps', 'aid', 'arw', 'awt', 'time1', 'time2', 'time3', 'time4', 'time5']


In [4]:
# Names of ‘variable’ and ‘value’ columns can be customized
df_cost = pd.melt(dfcost, id_vars =['lh', 'ps', 'id', 'wt', 'rw', 'alh', 'aps', 'aid', 'awt', 'arw'],
              var_name ='rep', value_name ='cost')
df_time = pd.melt(dftime, id_vars =['lh', 'ps', 'id', 'wt', 'rw', 'alh', 'aps', 'aid', 'awt', 'arw'],
              var_name ='rep', value_name ='time')

In [6]:
display((Markdown("### Statistics for Cost")))
df_cost.cost.describe()

### Statistics for Cost

count    80.000000
mean      0.531500
std       0.019623
min       0.480000
25%       0.510000
50%       0.540000
75%       0.540000
max       0.560000
Name: cost, dtype: float64